In [ ]:
!pip install scikit-learn

In [1]:
from collections import Counter
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

## Preprocessing for features
> Here we calculate the percentage change in 7 days for the ticker we need

In [2]:
def pre_process_labels(ticker):
    days_percent_change = 7
    df = pd.read_csv("main_csv_tickers_joined.csv", index_col=0)
    tickers = df.columns.values
    df.fillna(0, inplace = True)
    
    for i in range(1, days_percent_change + 1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
        
#     df.dropna(inplace = True)
    df.fillna(0, inplace = True)
#     print(tickers)
    return tickers, df

tickers, df = pre_process_labels('AXISBANK.NS')
# print(df.head(10))
df_test = df.iloc[1220:, 50:]
df_test.join(df['AXISBANK.NS'])

,AXISBANK.NS_1d,AXISBANK.NS_2d,AXISBANK.NS_3d,AXISBANK.NS_4d,AXISBANK.NS_5d,AXISBANK.NS_6d,AXISBANK.NS_7d,AXISBANK.NS
Date,,,,,,,,
2020-07-15,0.017227,0.015118,0.017227,0.045822,0.122583,0.080159,0.044416,426.649994
2020-07-16,-0.002074,0.000000,0.028111,0.103571,0.061866,0.026728,0.000000,434.000000
2020-07-17,0.002078,0.030247,0.105865,0.064073,0.028862,0.000000,0.000000,433.100006
2020-07-20,0.028111,0.103571,0.061866,0.026728,0.000000,0.000000,0.000000,434.000000
2020-07-21,0.073398,0.032833,-0.001345,0.000000,0.000000,0.000000,0.000000,446.200012
2020-07-22,-0.037791,-0.069631,0.000000,0.000000,0.000000,0.000000,0.000000,478.950012
2020-07-23,-0.033091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,460.850006
2020-07-24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,445.600006


## Generating Labels buy(1), sell(-1), hold(0)
> * The Requirement variable is to mention we wanna sell the stock if there's a decrease of that percent change in 7 days 

In [3]:
def buy_sell_hold(*args):
    cols = [c for c in args]
#     print(cols)
    requirement = 0.02 #2% change
    for col in cols:
        if col > requirement:
            return 1
        elif col < -requirement:
            return -1
    return 0

## Now we need to map the above function to all the percent change columns. We do this by using mapping

* ### We need to compare the percent change we calculated with the pct_change 

In [20]:
def extract_featuresets(ticker):
    tickers, df = pre_process_labels(ticker)
    
    df['{}_target'.format(ticker)] = list(map( buy_sell_hold,  *[df['{}_{}d'.format(ticker, i)]for i in range(1,8)] ))
    
    vals = df['{}_target'.format(ticker)].values
    str_vals = [str(i) for i in vals]
    print("data Spread:", Counter(str_vals))
    
#     df_test = df.iloc[:, 50:]
#     print(df_test)
#     if df.isnull().values.any:
#         print("hello nan")

    df.fillna(0, inplace = True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace = True)
    
    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace = True)
    
    X = df_vals.values
    y = df['{}_target'.format(ticker)].values
    
    return X, y, df
    
#     print(df_vals)
#     df_vals['AXISBANK.NS'].plot()

## We use KNN to classify into sell, buy or hold

In [23]:
def Knn(ticker):
    X, y, df = extract_featuresets(ticker)
    X_train, X_test, y_train , y_test = train_test_split(X, y, test_size = 0.33)
#     print(X_train.shape)
#     print(y_train.shape)


    clf_knn = neighbors.KNeighborsClassifier()
    clf_voting = VotingClassifier([('lsvc', svm.LinearSVC()),
                           ('knn', neighbors.KNeighborsClassifier()),
                           ('rfor', RandomForestClassifier())])
    
    
    clf_knn.fit(X_train, y_train)
    confidence = clf_knn.score(X_test, y_test)
    print('Accuracy by KNN : ', confidence)
    predictions = clf_knn.predict(X_test)
    print('prediction spread by KNN:', Counter(predictions)) 
    
    clf_voting.fit(X_train, y_train)
    confidence = clf_voting.score(X_test, y_test)
    print('Accuracy by VotingClassifier : ', confidence)
    predictions = clf_voting.predict(X_test)
    print('prediction spread by VotingClassifier:', Counter(predictions)) 
    
Knn('AXISBANK.NS')

data Spread: Counter({'1': 552, '-1': 538, '0': 138})
Accuracy by KNN :  0.4236453201970443
prediction spread by KNN: Counter({-1: 215, 1: 163, 0: 28})
Accuracy by VotingClassifier :  0.42610837438423643
prediction spread by VotingClassifier: Counter({-1: 206, 1: 200})
